# Encryption 2. 
### Byte by Byte encryption (XOR)
The idea here is:
- Not to use the character substitution (like in [Encryption 101](./Encryption.1.1.ipynb))
- The strings to use (message, and in some case, the key) would be converted into byte arrays. Their values will be be considered as numbers, disconnected from their character (ASCII) value.
- The byte array representing the message will be processed using the byte array representing the key, using logical opration(s) (like XOR, as seen below).
- Once back decrypted to its original value, the byte array will finally be returned to its string (aka character) value.

---

For clarity, we will start with a readable key (character key), and we'll move later to a numeric one.  
We start here with a key like `DAVID`, and encrypt a message with the same length, like `HELLO`.

---

Still for clarity, we will first process **_bit by bit_**

#### Bit by bit

First, the message.

In [1]:
String hello = "HELLO"; // Page 309
byte[] helloBytes = hello.getBytes();
StringBuilder binaryHello = new StringBuilder();
for (byte b : helloBytes) {
    binaryHello.append(String.format("%s ", Integer.toBinaryString(b)));
}
System.out.printf("Binary Hello (lpad) : [%s]\n", binaryHello.toString().trim());

binaryHello = new StringBuilder();
for (byte b : helloBytes) {
    binaryHello.append(String.format("%s ", Integer.toBinaryString(b)));
}
// Uppercase letters are 7 bit long.
System.out.printf("Binary Hello : [%s]\n", binaryHello.toString().trim());

Binary Hello (lpad) : [1001000 1000101 1001100 1001100 1001111]
Binary Hello : [1001000 1000101 1001100 1001100 1001111]


java.io.PrintStream@3c3070fd

Then, the Key

In [2]:
byte[] davidBytes = "DAVID".getBytes();
StringBuilder binaryDavid = new StringBuilder();
for (byte b : davidBytes) {
    binaryDavid.append(String.format("%s", Integer.toBinaryString(b)));
}
String davidKey = binaryDavid.toString().trim(); // Binary String

binaryHello = new StringBuilder();
for (byte b : helloBytes) {
    binaryHello.append(String.format("%s", Integer.toBinaryString(b)));
}

Let's display the current status of the data, in their _binary_ format, for the clarity of the next steps.

In [3]:
String helloBuffer = binaryHello.toString().trim();
System.out.println("Before proceeding:");
System.out.printf("Message: %s\nThe Key: %s\n", helloBuffer, davidKey);

Before proceeding:
Message: 10010001000101100110010011001001111
The Key: 10001001000001101011010010011000100


java.io.PrintStream@3c3070fd

Now the data are ready, here is how we are going to proceed.  
Bit by bit, (first bit of the message with the first bit of the key, second byte of the message with the second byte of the key, etc), we apply an XOR (Exclusive OR) logical operation. Here is what it means:  
- if the message bit is identical to the key byte, the result value is `0`.
- if the message bit and the key byte are different, the result value is `1`.

In [4]:
byte[] messageBits = helloBuffer.getBytes();
byte[] keyBits     = davidKey.getBytes();
List<Byte> encoded = new ArrayList<>();
// bit by bit
for (int i=0;i<messageBits.length; i++) {
    // It's an XOR
    encoded.add((messageBits[i] == keyBits[i]) ? (byte)'0' : (byte)'1');
}
byte[] newArray = new byte[messageBits.length];
for (int i=0; i<encoded.size(); i++) {  // TODO Any way to improve this loop ? Byte[] to byte[] ?
    newArray[i] = encoded.get(i);
}
String encodedString = new String(newArray);
System.out.printf("Encoded message:\n%s\n", encodedString);

Encoded message:
00011000000100001101000001010001011


java.io.PrintStream@3c3070fd

Now, let's see the decoding process.  
Let's re-display the original message, from its byte rerpresentation

In [5]:
// With the original message:
for (int i=0; i<helloBuffer.length(); i+=7) {
    int c = Integer.parseInt(helloBuffer.substring(i, i+7), 2);
    System.out.printf("Original - %s\n", (char)c);
}

Original - H
Original - E
Original - L
Original - L
Original - O


And now, we decrypt, apply the logical XOR, with the encoded message, and the key, bit by bit:

In [6]:
List<Byte> decoded = new ArrayList<>();
// bit by bit
for (int i=0;i<messageBits.length; i++) {
    // newArray contains the en coded message
    decoded.add((newArray[i] == keyBits[i]) ? (byte)'0' : (byte)'1');
}

newArray = new byte[messageBits.length];
for (int i=0; i<decoded.size(); i++) {  // Same as above, improve this loop
    newArray[i] = decoded.get(i);
}

Now we have the decoded bits, let's see them

In [7]:
String decodedString = new String(newArray);
System.out.printf("\nDecoded:\n%s\n\n", decodedString);


Decoded:
10010001000101100110010011001001111



java.io.PrintStream@3c3070fd

Those bits can be displayed as bytes or characters

In [8]:
// With the decoded message:
for (int i=0; i<helloBuffer.length(); i+=7) {
    int c = Integer.parseInt(helloBuffer.substring(i, i+7), 2);
    System.out.printf("Decoded - %s\n", (char)c);
}

Decoded - H
Decoded - E
Decoded - L
Decoded - L
Decoded - O


That's it! We're back to the original message.  
_Notice_: you can use any character, as we do not care about their ASCII value, upper case, lower case, accented, punctuation, averything will do.

#### Next step, byte by byte
Now we've seen - bit by bit - how this work, let's proceed byte by byte. Logical operations work on any scalar type.  
Also, we will this time process a message longer than the key. The key is "repeated" at will, the key is repeated as long as this repetion's length is lower than the length of the message to process. See the modulo operation, at `[i % davidBytes.length]`, in the code below.  

The `modulo` operation is going to be used several times in this section, and further ones. I've seen many definitions of it... My favorite one is "`X modulo Y` is the remainder of division of `X` by `Y`".  
For example, `145 modulo 12` is `1`.  
In Java (and many other languages as well), the `modulo` operation is represented by the `%` character.  

We will also be explicitely using the XOR operation.  
In Java, the logical AND is `&`, the logical OR is `|`, and the logical XOR is `^`, as seen below.

In [9]:
// davidBytes contains "DAVID", 5 bytes
String message02 = "Hello World! We do not depend on the key's length!";
final byte[] message02Bytes = message02.getBytes();
byte[] encoded02 = new byte[message02Bytes.length];
for (int i=0; i<message02Bytes.length; i++) {
    encoded02[i] = (byte)(message02Bytes[i] ^ davidBytes[i % davidBytes.length]);
}
String encodedString02 = new String(encoded02);
System.out.printf("Encoded, byte by byte: [%s]\n", encodedString02);

Encoded, byte by byte: [$:%+d9;( `v!d%9i*+5v-!4$8-d+/v=,!a=,=c2v%!*&"!e]


java.io.PrintStream@3c3070fd

The encoded string is notoriously un-readable...  
Now, we decode, byte by byte, and then into a string

In [10]:
// Try decoding the same way
byte[] toDecode = encodedString02.getBytes();
byte[] decoded02 = new byte[toDecode.length];
for (int i=0; i<toDecode.length; i++) {
    decoded02[i] = (byte)(toDecode[i] ^ davidBytes[i % davidBytes.length]);
}
String decodedString02 = new String(decoded02);
System.out.printf("Decoded, byte by byte: [%s]\n", decodedString02);

Decoded, byte by byte: [Hello World! We do not depend on the key's length!]


java.io.PrintStream@3c3070fd

### With an integer key
A Long would work as well. The only requirement is to be able to have o byte array representation of the key, just like we did above with s key of thw String type.

First we define a key, and get its byte array representation.

In [11]:
import java.nio.ByteBuffer;

int intKey = 123456789;
byte[] byteKey = ByteBuffer.allocate(4).putInt(intKey).array();

Then we define a message, and similarly, get ots byte array equivalent

In [12]:
String message03 = "Now the key is an int. Let's see if it still works...";
final byte[] message03Bytes = message03.getBytes();

We encode the message, using the byte array representation of the integer key, just like we did beflre with the `DAVID` key, and we display the values the each byte of the encoded result

In [13]:
import java.util.stream.Collectors;


byte[] encoded03 = new byte[message03Bytes.length];
for (int i=0; i<message03Bytes.length; i++) {
    encoded03[i] = (byte)(message03Bytes[i] ^ byteKey[i % byteKey.length]);
}
ByteArrayOutputStream intEncoded = new ByteArrayOutputStream();
intEncoded.writeBytes(encoded03);

// The encoded data is a Byte array, NOT a String
List<Byte> encodedByteList = new ArrayList<>();
for (byte b : encoded03) {
    encodedByteList.add(b);
}
System.out.printf("Encoded, byte by byte, with an int key: [%s]\n", 
                  encodedByteList.stream().map(Integer::toString).collect(Collectors.joining(", ")));

Encoded, byte by byte, with an int key: [73, 52, -70, 53, 115, 51, -88, 53, 108, 62, -76, 53, 110, 40, -19, 116, 105, 123, -92, 123, 115, 117, -19, 89, 98, 47, -22, 102, 39, 40, -88, 112, 39, 50, -85, 53, 110, 47, -19, 102, 115, 50, -95, 121, 39, 44, -94, 103, 108, 40, -29, 59, 41]


java.io.PrintStream@3c3070fd

Following the exact same pattern, we now decode

In [14]:
// Try decoding the same way
byte[] decoded03 = new byte[encoded03.length];
for (int i=0; i<encoded03.length; i++) {
    decoded03[i] = (byte)(encoded03[i] ^ byteKey[i % byteKey.length]);
}
String decodedString03 = new String(decoded03);
System.out.printf("Decoded, byte by byte, with an int key: [%s]\n", decodedString03);


Decoded, byte by byte, with an int key: [Now the key is an int. Let's see if it still works...]


java.io.PrintStream@3c3070fd

Sounds good!  
To facilitate the next steps, we will create encryption and decryption methods.

In [15]:
public class EncryptionUtils {
    public static ByteArrayOutputStream intEncode(byte[] messageBytes, int key) {
        byte[] byteKey = ByteBuffer.allocate(4).putInt(key).array();
        byte[] encoded = new byte[messageBytes.length];
        for (int i=0; i<messageBytes.length; i++) {
            encoded[i] = (byte)(messageBytes[i] ^ byteKey[i % byteKey.length]);
        }
        ByteArrayOutputStream intEncoded = new ByteArrayOutputStream();
        intEncoded.writeBytes(encoded);
    
        return intEncoded;
    }
    
    public static ByteArrayOutputStream intEncode(String message, int key) {
        final byte[] messageBytes = message.getBytes();
        return intEncode(messageBytes, key);
    }
    
    public static ByteArrayOutputStream intDecode(ByteArrayOutputStream baos, int key) {
        byte[] byteKey = ByteBuffer.allocate(4).putInt(key).array();
        byte[] encoded = baos.toByteArray();
        byte[] decoded = new byte[encoded.length];
        for (int i=0; i<encoded.length; i++) {
            decoded[i] = (byte)(encoded[i] ^ byteKey[i % byteKey.length]);
        }
        ByteArrayOutputStream intDecoded = new ByteArrayOutputStream();
        intDecoded.writeBytes(decoded);
        return intDecoded;
    }
}

Notice that those guys are using `ByteArrayOutputStream` to store the byte data.  
Let's give it a try, with a French message, containing punctuation, accented characters, etc.

In [16]:
int newKey = 98765432;
ByteArrayOutputStream encodedBAOS = EncryptionUtils.intEncode("On essaye avec une méthode dédiée ?", newKey);
System.out.println("Encoded done...");
String decodedBAOS = new String(EncryptionUtils.intDecode(encodedBAOS, newKey).toByteArray());
System.out.printf("Decoded: [%s]\n", decodedBAOS);

Encoded done...
Decoded: [On essaye avec une méthode dédiée ?]


java.io.PrintStream@3c3070fd

That seems to work...  
Let's see, with this XOR function, how the process described [previously](./Encryption.101.ipynb#A-Test) behaves...

In [17]:
System.out.println("\nAlice - Bernard ping-pong, new test...");
int aliceIntKey   = 12345678;
int bernardIntKey = 98765432;

String bingBongMessage = "Bing Bong Message !";
ByteArrayOutputStream stepOne = EncryptionUtils.intEncode(bingBongMessage, aliceIntKey);         // Encode with Alice's key
ByteArrayOutputStream stepTwo = EncryptionUtils.intEncode(stepOne.toByteArray(), bernardIntKey); // Encode with Bernard's key
ByteArrayOutputStream stepThree = EncryptionUtils.intDecode(stepTwo, aliceIntKey);               // Decode with Alice's key
ByteArrayOutputStream stepFour = EncryptionUtils.intDecode(stepThree, bernardIntKey);            // Final result: Decode with Bernard's key

String decodedBingBong = new String(stepFour.toByteArray());
System.out.printf("Finally: [%s]\n", decodedBingBong);


Alice - Bernard ping-pong, new test...
Finally: [Bing Bong Message !]


java.io.PrintStream@3c3070fd

Wow! The XOR approach seems to fit this context!  
The only thing you could reproach this method with is the number of trips the message has to do...  
This is going to be addressed later.